\begin{center} 
Chun-Yuan (Scott) Chiu 
\end{center}
\begin{center} 
chunyuac@andrew.cmu.edu 
\end{center}

# 1. {-}

For completeness we list Form 1 here: 
\begin{align*}
&\text{Maximize } M\\
&\text{Subject to } \sum_{i=1}^n\epsilon_i \le C, \epsilon_i\ge 0, y_i(x_i^T\beta+\beta_0) \ge M(1-\epsilon_i), \lVert\beta\rVert_2=1. \qquad \text{(Form 1)}
\end{align*}

## (a) {-}

Assume $b$ is positive. 
Replacing $\beta$ by $b\beta$ and $\beta_0$ by $b\beta_0$ the inequality 
\begin{align*}
y_i(x_i^T\beta + \beta_0) \ge M(1-\epsilon_i)\lVert\beta\rVert_2, 
\end{align*}
we obtain 
\begin{align*}
& y_i(x_i^T(b\beta) + b\beta_0) \ge M(1-\epsilon_i)\lVert b\beta\rVert_2 \\
\iff& by_i(x_i^T\beta + \beta_0) \ge b M(1-\epsilon_i)\lVert \beta\rVert_2 \\
\iff& y_i(x_i^T\beta + \beta_0) \ge M(1-\epsilon_i)\lVert \beta\rVert_2, 
\end{align*}
which is the original inequality. Thus scaling the coefficients of the equation of the decision boundary does not do anything to the inequality. 


Given this observation we choose $\lVert\beta\rVert_2$ to be $1/M$, rewriting the optimization problem as 
\begin{align*}
&\text{Minimize } \lVert\beta\rVert_2\\
&\text{Subject to } \sum_{i=1}^n\epsilon_i \le C, \epsilon_i\ge 0, y_i(x_i^T\beta+\beta_0) \ge (1-\epsilon_i). \qquad \text{(Form 2)}
\end{align*}
Here minimizing $\lVert\beta\rVert_2 = 1/M$ is the same as maximizing $M$. 
This is equivalent to replacing $\beta$ by $\beta/M$ and $\beta_0$ by $\beta_0/M$ in Form 1. The information about the width of the margin is now embedded in the coefficients $\beta$ and $\beta_0$ in Form 2. 


## (b) {-}

Form 3 is as follows
\begin{align*}
&\text{Minimize } \lVert\beta\rVert_2 + D\sum_{i=1}^n\epsilon_i\\
&\text{Subject to } \epsilon_i\ge 0, y_i(x_i^T\beta+\beta_0) \ge (1-\epsilon_i). \qquad \text{(Form 3)}
\end{align*}
Big $C$ in Form 2 corresponds to small penalty for misclassifications, which corresponds to small $D$ in Form 3. 
